# Inference notebook for "Cassava leaf disease classification" competition

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.image as mpimg

In [3]:
# BASE PATHS: ONLY THINGS TO CHANGE WHEN SHARING THIS NOTEBOOK
BASE_DIR = Path("../input/cassava-leaf-disease-classification") #Path to data directory
MODELS_DIR = Path("../input/cassavamodels") #Path to saved models
IMAGE_DIR = Path(BASE_DIR, "train_images") #Path to images directory
OUTPUT_DIR = Path("./") #Path to 'output' directory

SEED = 117
IMAGE_SIZE = 512

In [4]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy) #shortens training time by 2x

In [5]:
import tensorflow as tf
from keras.models import load_model
from keras.layers import Dropout
from keras.preprocessing.image import load_img, img_to_array
import albumentations as alb


sample = pd.read_csv(Path(BASE_DIR, "sample_submission.csv"))

In [6]:
from tqdm import trange

# For TTA
def tta_predict(image, model, augments, preprocess_func, N=5, kfold=False):
    preds = []
    for i in range(N):
        augment_image = augments(image=image)["image"]
        augment_image = preprocess_func(image)
        augment_image = np.expand_dims(augment_image, axis=0)
        preds.append(model.predict(augment_image))
    
    if kfold:
        return np.mean(preds, axis=0)
    return np.argmax(np.mean(preds, axis=0))


# # For Monte-Carlo dropout
def predict_proba(image, model, num_samples):
    preds = [model.predict(image) for _ in trange(num_samples)]

    return np.stack(preds).mean(axis=0)
     
def predict_class(image, model, num_samples):
    proba_preds = predict_proba(image, model, num_samples)

    return np.argmax(proba_preds)

## Single model inference

In [7]:
# model = load_model(Path(MODELS_DIR, "EfficientNetB0_512.h5"))

# mc = False #Monte-Carlo dropout
# tta = not(mc)
# tta_augments = alb.Compose([
#     alb.ShiftScaleRotate(p=0.5),
#     alb.CoarseDropout(p=0.5),
#     alb.HorizontalFlip(p=0.5),
#     alb.VerticalFlip(p=0.5),
#     alb.RandomCrop(height=IMAGE_SIZE, width=IMAGE_SIZE, p=0.5)
# ])

# center = alb.CenterCrop(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0)

In [8]:
# results = []
# for image_id in sample.image_id:
#     image = load_img(Path(BASE_DIR, "test_images", image_id)) #Read images
#     image = img_to_array(image)
    
#     if tta:
#         results.append(tta_predict(image, model, tta_augments, efficient_preprocess, K=5))
#     elif mc:
#         image = center(image=image)["image"]
# #         image = xception_preprocess(image)
#         image = np.expand_dims(image, axis=0)
#         results.append(predict_class(image, model, 5))
#     else:
#         image = center(image=image)["image"]
# #         image = xception_preprocess(image)
#         image = np.expand_dims(image, axis=0)
#         results.append(np.argmax(model.predict(image)))

# sample['label'] = results
# sample.to_csv("submission.csv", index=False)
# display(sample)

## K-folds inference

In [9]:
def kfolds_predict(image, models, augments, preprocess_func, tta=False, ensemble=False):
    preds = []
    for model in models:
        if tta:
            pred = tta_predict(image, model, augments, preprocess_func, 5, True)
        else:
            augment_image = augments(image=image)["image"]
            augment_image = preprocess_func(augment_image)
            augment_image = np.expand_dims(augment_image, axis=0)
            pred = model.predict(augment_image)
        preds.append(pred)
        
    if ensemble:
        return np.mean(preds, axis=0)
    return np.argmax(np.mean(preds, axis=0))


def ensemble_predict(image, models, augments, funcs, tta=False):
    preds = []
    for name in models.keys():
        pred = kfolds_predict(image, models[name], augments, funcs[name], tta, True)
        preds.append(pred)
    
    return np.argmax(np.mean(preds, axis=0))

In [10]:
K = 3
models_names = ["Xception", "EfficientNetB0"]
models_dict = {"Xception":[], "EfficientNetB0":[]}
funcs = {
    "Xception":tf.keras.applications.xception.preprocess_input,
    "EfficientNetB0":tf.keras.applications.efficientnet.preprocess_input
}
# models = []
for name in models_names:
    for i in range(1, K+1):
        model = load_model(Path(MODELS_DIR, "{0:s}_512_fold{1:d}.h5".format(name, i)))
#         models.append(model)
        models_dict[name].append(model)
    
center = alb.CenterCrop(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0)
resize = alb.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0)

tta_augments = alb.Compose([
    alb.ShiftScaleRotate(p=0.5),
    alb.CoarseDropout(p=0.5),
    alb.HorizontalFlip(p=0.5),
    alb.VerticalFlip(p=0.5),
    alb.RandomCrop(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0)
#     alb.Resize(height=IMAGE_SIZE, width=IMAGE_SIZE, p=1.0)
])

In [11]:
import matplotlib.pyplot as plt

tta = False
if tta:
    augments = tta_augments
else:
    augments = center
    
results = []
for image_id in sample.image_id:
    preds = []
    image = load_img(Path(BASE_DIR, "test_images", image_id)) #Load images
    image = img_to_array(image)
    
#     results.append(kfolds_predict(image, models, augments, preprocess_func, tta))
    results.append(ensemble_predict(image, models_dict, augments, funcs, tta))

sample['label'] = results
sample.to_csv(Path(OUTPUT_DIR, "submission.csv"), index=False)
display(sample)

,image_id,label
0,2216849948.jpg,4
